In [65]:
import os
import glob
import json
import shutil
import urllib

from collections import defaultdict
import xml.etree.ElementTree as ET

# Download the original dataset

In [70]:
URL = 'https://www.rondhuit.com/download/livedoor-news-data.tar.gz'

if not os.path.isdir('livedoor-news-data'):
    
    if not os.path.isfile('livedoor-news-data.tar.gz'):
        urllib.request.urlretrieve(URL, 'livedoor-news-data.tar.gz')
        
    shutil.unpack_archive(
        'livedoor-news-data.tar.gz', 'livedoor-news-data')


glob.glob('livedoor-news-data/*')

['livedoor-news-data/sports-watch.xml',
 'livedoor-news-data/smax.xml',
 'livedoor-news-data/movie-enter.xml',
 'livedoor-news-data/peachy.xml',
 'livedoor-news-data/it-life-hack.xml',
 'livedoor-news-data/kaden-channel.xml',
 'livedoor-news-data/topic-news.xml',
 'livedoor-news-data/livedoor-homme.xml',
 'livedoor-news-data/dokujo-tsushin.xml']

# Explore the xml format

In [17]:
root = ET.parse('livedoor-news-data/sports-watch.xml').getroot()
root

<Element 'add' at 0x7fab18bd8470>

In [18]:
root[:3]

[<Element 'doc' at 0x7fab18bd8530>,
 <Element 'doc' at 0x7fab18bd8a10>,
 <Element 'doc' at 0x7fab18be00b0>]

In [21]:
root[0][:3]

[<Element 'field' at 0x7fab18bd85f0>,
 <Element 'field' at 0x7fab18bd8650>,
 <Element 'field' at 0x7fab18bd86b0>]

In [33]:
[(f.get('name'), f.text) for f in root[0][:9]]

[('url', 'http://news.livedoor.com/article/detail/4597641/'),
 ('cat', 'sports-watch'),
 ('date', '2010-02-10T01:50:00Z'),
 ('title', '【Sports Watch】秋山成勲、メールで吉田に対戦迫った!?'),
 ('body', '今月8日、都内ホテルでは、総合格闘家・吉田秀彦の引退試合興行「ASTRA」の開催が発表された。'),
 ('body', None),
 ('body',
  'バルセロナ五輪柔道金メダリストとしての実績を引っさげ、2002年にプロ総合格闘家に転向。以後、数々の死闘を繰り広げてきた吉田。昨年大晦日のDynamite!!では、石井慧との金メダリスト対決を制し、4月に迎える引退試合の相手には、桜庭和志やまさかの朝青龍といった報道が駆け巡る中、“反骨の柔道王”秋山成勲が吉田にメールで対戦を迫っていたというのだ。'),
 ('body', None),
 ('body',
  '会見翌日の9日に更新された秋山成勲オフィシャルブログでは、「吉田秀彦対秋山成勲」と題し、「常に憧れ目標にしてきた吉田先輩が引退。正直寂しい気持ちはありますが、ほんまにお疲れ様でした！引退試合はもちろん自分とやるでしょ！？」とストレートに書き綴りながらも、「さっき吉田先輩にメールで自分とやるでしょ？的なメールを打って、軽く流されましたが」と、実際に吉田へ対戦を打診をしていたことを明かした。')]

In [46]:
def to_json_doc(xml_doc):
    json_map = defaultdict(list)
    
    for field in xml_doc:
        text = field.text if field.text else ''
        json_map[field.get('name')].append(text)
        
        
    return {k: '\n'.join(v) for k,v in json_map.items()}
        
    
to_json_map(root[0])

{'url': 'http://news.livedoor.com/article/detail/4597641/',
 'cat': 'sports-watch',
 'date': '2010-02-10T01:50:00Z',
 'title': '【Sports Watch】秋山成勲、メールで吉田に対戦迫った!?',
 'body': '今月8日、都内ホテルでは、総合格闘家・吉田秀彦の引退試合興行「ASTRA」の開催が発表された。\n\nバルセロナ五輪柔道金メダリストとしての実績を引っさげ、2002年にプロ総合格闘家に転向。以後、数々の死闘を繰り広げてきた吉田。昨年大晦日のDynamite!!では、石井慧との金メダリスト対決を制し、4月に迎える引退試合の相手には、桜庭和志やまさかの朝青龍といった報道が駆け巡る中、“反骨の柔道王”秋山成勲が吉田にメールで対戦を迫っていたというのだ。\n\n会見翌日の9日に更新された秋山成勲オフィシャルブログでは、「吉田秀彦対秋山成勲」と題し、「常に憧れ目標にしてきた吉田先輩が引退。正直寂しい気持ちはありますが、ほんまにお疲れ様でした！引退試合はもちろん自分とやるでしょ！？」とストレートに書き綴りながらも、「さっき吉田先輩にメールで自分とやるでしょ？的なメールを打って、軽く流されましたが」と、実際に吉田へ対戦を打診をしていたことを明かした。\n\nもちろん、階級が違う上、秋山は米UFCを主戦場にしている現状、対戦することはまずない。それでも、我が道を行き、空気を一切読まない秋山だけに、案外本気に考えていた可能性もなきにしもあらず——。そんなブログの最後には、「ほんまに柔道から格闘技の道を作って頂いたパイオニアだと自分は思い尊敬してます！引退試合頑張ってください！」と吉田にエールを送る秋山であった。'}

# Transform the data

In [56]:
for xml_file in glob.glob('livedoor-news-data/*'):
    
    xml_root = ET.parse(xml_file).getroot()
    json_docs = [to_json_doc(doc) for doc in xml_root]
    
    json_file = xml_file.replace('livedoor-news-data/', '').replace('.xml', '.json')
    with open(json_file, 'w') as f:
        json.dump(json_docs, f)
    
glob.glob('./*.json')

['./peachy.json',
 './smax.json',
 './kaden-channel.json',
 './it-life-hack.json',
 './dokujo-tsushin.json',
 './sports-watch.json',
 './movie-enter.json',
 './livedoor-homme.json',
 './topic-news.json']

In [59]:
with open('topic-news.json') as f:
    docs = json.load(f)
    
docs[:3]

[{'url': 'http://news.livedoor.com/article/detail/5903225/',
  'cat': 'topic-news',
  'date': '2011-10-02T01:00:00Z',
  'title': '悪評が次から次へと溢れ出る川越シェフ',
  'body': '最近、TVでも人気者の川越シェフ。代官山のおしゃれなイタリアンレストランのオーナーシェフでありながら、タレントとも軽妙なトークを交わすマルチな才能で、TV業界でもその地位を不動のものとしつつある。また、その甘いマスクに魅せられた、「川越女子」なる固定ファンまで存在するという。\n\nしかし、ここにきて川越シェフの悪評がどんどん噴出しているようだ。最近では、ミニストップの川越シェフプロデュースクリスマスケーキが、1万円という超高額で売り出されることが話題となった。\n\n【関連記事】\nミニストップの川越シェフプロデュースクリスマスケーキがバカ高いと話題に\nhttp://news.livedoor.com/topics/detail/5899457/\n\n普通のケーキなら、原価は200円程度。川越シェフプロデュースだけに、多少豪華に作ってあるかもしれないが、それでも「ぼったくり」というネット上の反応が多かった。また、それ以上に、「有名になった途端、ここぞとばかりに金儲けしている」という声も散見された。\n\nまた、普段のマスコミ対応の悪さもかなりのものだ。数年前までは、川越シェフなどほとんど誰も知らないような状態だったが、有名となった今は手の平を返すように、週刊誌の記者なを見下すような態度が見られるようになったという。\n\nマスコミのおかげで有名になっておきながら、恩知らずとはこのことだろう。\n\n【関連記事】\nシェフのタレント化はもうたくさん!?\u3000マスコミが嫌うタレントシェフ\nhttp://news.livedoor.com/topics/detail/5801160/\n\nしかし、単なるシェフ風情が調子に乗ってしまうのも無理はない。何せ、「株式会社TATSUYA KAWAGOE」の年商は約9,000万円、さらにTVへの出演料も高額に上る。\n\nまた、6月にTV番組で「バツ2」の独身であることを告白している。あの甘いマスク